# Nearest neighbors

Our first learning algorithm is conceptually simple: Given a new point to classify, survey the nearest examples and choose the most frequent class. This is called the **$k$ nearest neighbors** (KNN) algorithm, where $k$ is the number of neighboring examples to survey.

## Norms

The existence of "closest" examples means that we need to define a notion of distance in spaces of any dimension. Let $\real^d$ be the space of vectors with $d$ real components, and let $\bfzero$ be the vector of all zeros.

```{prf:definition}
A **norm** is a function $\|\bfx\|$ on $\real^d$ that satisfies the following properties:

$$
\|\bfzero\| &= 0,  \\ 
\|\bfx\| &> 0 \text{ if $\bfx$ is a nonzero vector}, \\ 
\|c\bfx\| &= |c| \, \|x\| \text{ for any real number $c$}, \\ 
\|\bfx + \bfy \| &\le \bfx + \bfy.
$$
```

The last inequality above is called the **triangle inequality**. It turns out that these four characteristics are all we expect from a function that behaves like a distance. 

We will encounter two different norms:

* The 2-norm, $\|\bfx\|_2 = \bigl(x_1^2 + x_2^2 + \cdots + x_d^2\bigr)^{1/2}.$
* The 1-norm, $\|\bfx\|_1 = |x_1| + |x_2| + \cdots + |x_d|.$

On the number line (i.e., $\real^1$), the distance between two values is just the absolute value of their difference, $|x-y|$. In $\real^d$, the distance between two vectors is the norm of their difference, $\| \bfx - \bfy \|$. 

The 2-norm is also called the *Euclidean norm*. It generalizes ordinary geometric distance in $\real^2$ and $\real^3$ and is usually considered the default. The 1-norm is sometimes called the *Manhattan norm*, because in $\real^2$ it represents the total number of east/west and north/south moves needed between points on a grid.

## Algorithm

As data, we are given labeled examples $\bfx_1,\ldots,\bfx_n$ in $\real^d$. Given a new query vector $\bfx$, find the $k$ labeled vectors closest to $\bfx$ and choose the most frequently occurring label among them. Ties can be broken randomly.

KNN divides up the feature space into domains that are dominated by nearby instances. The boundaries between those domains (called **decision boundaries**) can be fairly complicated, though, as shown in the animation below. 

```{raw} html
<video width=640 controls src="../_static/knn_demo.mp4"></video>
```

Implementation of KNN is straightforward for small data sets, but requires care to get reasonable execution efficiency for large sets.

## KNN in sklearn

Let's revisit the penguins. We use `dropna` to drop any rows with missing values.

In [1]:
import seaborn as sns
import pandas as pd
penguins = sns.load_dataset("penguins")
penguins = penguins.dropna()
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


The data set has four quantitative columns that we use as features, and the species name is the label.

In [2]:
col = ["bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]
X = penguins[col]
y = penguins["species"]

Scikit-learn plays nicely with pandas, so we don't have to translate the data into new structures.

In [3]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X,y)

KNeighborsClassifier()

We can manually find the neighbors of a new vector. However, we have to make the query in the form of a data frame, since that is how the training data was provided. Here we make a query frame for values very close to the ones in the first row of the data.

In [4]:
query = pd.DataFrame([[39,19,180,3750]],columns=X.columns)
dist,idx = knn.kneighbors(query)
idx[0]

array([  0, 143,  53, 100, 153])

As you see above, the first point (index 0) was the closest, followed by four others. We can look up the labels of these points:

In [5]:
y[idx[0]]

0         Adelie
143       Adelie
53        Adelie
100       Adelie
153    Chinstrap
Name: species, dtype: object

By a vote of 4–1, the classifier should choose Adelie as the result at this location.

In [6]:
knn.predict(query)

array(['Adelie'], dtype=object)

Note that some data points can be outvoted by their neighbors.

In [7]:
print("Predicted:")
print(knn.predict(X.loc[:5,:]))

print("\nData:")
print(y[:5].values)

Predicted:
['Adelie' 'Adelie' 'Chinstrap' 'Adelie' 'Chinstrap']

Data:
['Adelie' 'Adelie' 'Adelie' 'Adelie' 'Adelie']


Here we split into training and test sets to gauge the performance of the classifier The `classification_report` function creates a summary of some of the important metrics.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size=0.2)
knn.fit(X_tr,y_tr)

yhat = knn.predict(X_te)
print(confusion_matrix(y_te,yhat))
print(classification_report(y_te,yhat))

[[25  4  1]
 [11  5  0]
 [ 2  0 19]]
              precision    recall  f1-score   support

      Adelie       0.66      0.83      0.74        30
   Chinstrap       0.56      0.31      0.40        16
      Gentoo       0.95      0.90      0.93        21

    accuracy                           0.73        67
   macro avg       0.72      0.68      0.69        67
weighted avg       0.73      0.73      0.72        67



<!-- To assess performance, let's apply 10-fold cross-validation to KNN learners with varying $k$.
```{code-cell}
from sklearn.model_selection import cross_val_score,KFold

K = range(1,10)
score_mean,score_std = [],[]
kf = KFold(n_splits=10,shuffle=True,random_state=1)
for k in K:
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn,X,y,cv=kf)
    score_mean.append(scores.mean())
    score_std.append(scores.std())

pd.DataFrame({"k":K,"accuracy mean":score_mean,"accuracy std":score_std})
```

There is no improvement here over $k=1$, in which each query just adopts the species of the nearest data vector. -->

The default norm in the KNN learner is the 2-norm. To use the 1-norm instead, add `metric="manhattan"` to the classifier construction call.

## Standardization

The values in the columns of the penguin frame are scaled quite differently. In particular, the values in the body mass column are more than 20x larger than the other columns on average. Consequently, the mass feature will dominate the distance calculations.

In [9]:
X.mean()

bill_length_mm         43.992793
bill_depth_mm          17.164865
flipper_length_mm     200.966967
body_mass_g          4207.057057
dtype: float64

To remedy this issue, we should transform the data into z-scores:

In [10]:
Z = X.transform( lambda x: (x-x.mean())/x.std() )

In this application, standardization makes performance dramatically better.

In [11]:
Z_tr, Z_te, y_tr, y_te = train_test_split(Z,y,test_size=0.2)
knn.fit(Z_tr,y_tr)

yhat = knn.predict(Z_te)
print(confusion_matrix(y_te,yhat))
print(classification_report(y_te,yhat))

[[30  0  0]
 [ 1 14  0]
 [ 0  0 22]]
              precision    recall  f1-score   support

      Adelie       0.97      1.00      0.98        30
   Chinstrap       1.00      0.93      0.97        15
      Gentoo       1.00      1.00      1.00        22

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67



## Pipelines

One inconvenience of the standardization step above is that it must be performed for any new data vector that comes along. Moreover, that standardization has to use the mean and std from our original creation of `Z`, so those values need to be tracked. 

The sklearn answer to this need is to create a **pipeline** that includes the transformation. Pipelines make it fairly easy to chain together data transformations, followed by a learner. The composite object acts like the original learner.

As you might guess, standardization of data is so common that it is predefined.

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler   # converts to z-scores

X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size=0.2)

knn = neighbors.KNeighborsClassifier(n_neighbors=5)
pipe = make_pipeline(StandardScaler(),knn)

pipe.fit(X_tr,y_tr)
pipe.score(X_te,y_te)

1.0

We can look under the hood a bit. The mean and variance of each of the original data columns is stored in the first part of the pipeline.

In [13]:
print(pipe[0].mean_)
print(pipe[0].var_)

[  44.1018797    17.10150376  201.11278195 4196.14661654]
[3.04095077e+01 3.77383233e+00 1.98408333e+02 6.64855922e+05]
